In [1]:
print("hello")

hello


In [2]:
import pandas as pd
import numpy as np

# Load the original dataset
df = pd.read_csv('Crop_recommendation.csv')
print(f"Original shape: {df.shape}")

target_size = 5000
num_classes = df['label'].nunique()
target_per_class = (target_size // num_classes) + 1  # 455 to slightly overshoot

noise_factor = 0.02  # 2% noise for subtle augmentation
numerical_cols = ['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall']

augmented_data = []  # List to hold dict representations of rows
grouped = df.groupby('label')

for label, group in grouped:
    # Add original rows
    for _, row in group.iterrows():
        augmented_data.append(row.to_dict())
    
    num_original = len(group)
    num_to_add = target_per_class - num_original
    
    # Generate augmented rows
    for _ in range(num_to_add):
        # Randomly sample a base row from the group
        idx = np.random.randint(0, num_original)
        base_row = group.iloc[idx]
        new_row_dict = base_row.to_dict()
        
        # Add noise to numerical features
        for col in numerical_cols:
            std = abs(base_row[col]) * noise_factor if abs(base_row[col]) > 0 else 1.0
            noise = np.random.normal(0, std)
            if col in ['N', 'P', 'K']:
                new_row_dict[col] = max(0, round(base_row[col] + noise))
            else:
                new_row_dict[col] = base_row[col] + noise
        
        augmented_data.append(new_row_dict)

# Create augmented DataFrame
aug_df = pd.DataFrame(augmented_data)

# Trim to exactly target_size if overshot
if len(aug_df) > target_size:
    aug_df = aug_df.sample(n=target_size, random_state=42).reset_index(drop=True)

print(f"Augmented shape: {aug_df.shape}")

# Save the augmented dataset
aug_df.to_csv('augmented_Crop_recommendation.csv', index=False)

Original shape: (2200, 8)
Augmented shape: (5000, 8)


In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pickle

# Train Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Evaluate
rf_train_pred = rf_model.predict(X_train)
rf_test_pred = rf_model.predict(X_test)
print("RF Training Accuracy:", accuracy_score(y_train, rf_train_pred))
print("RF Test Accuracy:", accuracy_score(y_test, rf_test_pred))

# Save the model
with open('rf_model.pkl', 'wb') as f:
    pickle.dump(rf_model, f)

NameError: name 'X_train' is not defined